In [33]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_squared_error,r2_score,roc_curve,auc
from sklearn.model_selection import KFold

import statsmodels.formula.api as smf
import statsmodels.api as sm


from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
import itertools as it

#Libraries for visualizing trees
from sklearn.tree import export_graphviz
from six import StringIO
from IPython.display import Image
import pydotplus
import time as time

In [34]:
data = pd.read_csv('data4.csv')

In [35]:
data = data.drop(columns = ['id', 'name', 'description', 'neighborhood_overview', 'host_id', 'host_name', 'host_neighbourhood', 'host_verifications', 'neighbourhood', 'bathrooms', 'bathrooms_text', 'amenities'])

In [36]:
data = data.drop(columns = ['review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value'])

In [37]:
data['y'] = data['review_scores_rating'] >= data['review_scores_rating'].median()
data['y'] = data.y.astype(int)

In [38]:
data.head()

,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,property_type,room_type,accommodates,bedrooms,beds,...,Hair dryer,Stove,Breakfast,Private patio or balcony,Full kitchen,Lockbox,Security cameras on property,Hot tub,Heating,y
0,100.0,84.0,1.0,1.0,Hyde Park,Private room in condominium (condo),Private room,1,1.0,1.0,...,1,0,1,0,0,1,0,0,1,1
1,100.0,97.0,1.0,1.0,West Town,Entire rental unit,Entire home/apt,2,1.0,1.0,...,1,1,0,0,0,0,0,0,0,0
2,94.0,94.0,1.0,1.0,Lincoln Park,Entire rental unit,Entire home/apt,4,2.0,2.0,...,1,1,0,0,0,0,0,0,1,0
3,NaN,100.0,0.0,1.0,Lincoln Park,Room in boutique hotel,Private room,2,1.0,2.0,...,1,0,0,0,0,0,0,0,1,1
4,NaN,0.0,0.0,1.0,West Town,Entire rental unit,Entire home/apt,4,2.0,2.0,...,1,1,0,0,0,1,1,0,1,0


In [39]:
#dropping low variance and low correlation predictors
data = data.drop(columns = ['beds', 'essentials', 'Hangers', 'wine glasses'])

additional cleaning:

Bathrooms (change to number, drop strings)

Neighborhood cleansed (merge levels)

property type (merge levels)

In [40]:
data = data.drop(data.index[data['Bathrooms'].isin(['Shared', 'Private', 'Half-bath'])])

In [41]:
data["Bathrooms"] = pd.to_numeric(data["Bathrooms"])

In [42]:
cutoff = data['neighbourhood_cleansed'].value_counts() <= 100


mask_obs = data['neighbourhood_cleansed'].value_counts()[cutoff].index
mask_dict = dict.fromkeys(mask_obs, 'other')

data['neighbourhood_cleansed'] = data['neighbourhood_cleansed'].replace(mask_dict)

In [43]:
data['neighbourhood_cleansed'].value_counts()

other                 1589
Near North Side        760
West Town              709
Lake View              561
Near West Side         407
Logan Square           397
Loop                   371
Lincoln Park           276
Uptown                 193
Near South Side        183
Lower West Side        176
Irving Park            153
Edgewater              145
Bridgeport             130
Avondale               128
Rogers Park            122
North Center           120
East Garfield Park     116
Name: neighbourhood_cleansed, dtype: int64

In [44]:
cutoff = data['property_type'].value_counts() <= 100


mask_obs = data['property_type'].value_counts()[cutoff].index
mask_dict = dict.fromkeys(mask_obs, 'other')

data['property_type'] = data['property_type'].replace(mask_dict)

In [45]:
data['property_type'].value_counts()

Entire rental unit                     3093
Entire condominium (condo)              723
Private room in rental unit             619
other                                   530
Private room in residential home        469
Entire residential home                 379
Private room in condominium (condo)     202
Entire guest suite                      171
Entire serviced apartment               122
Room in boutique hotel                  119
Entire loft                             109
Name: property_type, dtype: int64

In [46]:
data

,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,property_type,room_type,accommodates,bedrooms,price,...,Hair dryer,Stove,Breakfast,Private patio or balcony,Full kitchen,Lockbox,Security cameras on property,Hot tub,Heating,y
0,100.0,84.0,1.0,1.0,other,Private room in condominium (condo),Private room,1,1.0,95.0,...,1,0,1,0,0,1,0,0,1,1
1,100.0,97.0,1.0,1.0,West Town,Entire rental unit,Entire home/apt,2,1.0,65.0,...,1,1,0,0,0,0,0,0,0,0
2,94.0,94.0,1.0,1.0,Lincoln Park,Entire rental unit,Entire home/apt,4,2.0,127.0,...,1,1,0,0,0,0,0,0,1,0
3,NaN,100.0,0.0,1.0,Lincoln Park,Room in boutique hotel,Private room,2,1.0,329.0,...,1,0,0,0,0,0,0,0,1,1
4,NaN,0.0,0.0,1.0,West Town,Entire rental unit,Entire home/apt,4,2.0,105.0,...,1,1,0,0,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6539,100.0,100.0,0.0,1.0,West Town,Entire rental unit,Entire home/apt,2,1.0,53.0,...,0,0,0,0,0,0,0,0,0,0
6540,100.0,100.0,1.0,1.0,West Town,Entire condominium (condo),Entire home/apt,6,3.0,105.0,...,0,0,0,0,0,0,0,0,0,0
6541,NaN,NaN,0.0,1.0,Lake View,Entire rental unit,Entire home/apt,6,2.0,150.0,...,0,0,0,0,0,0,0,0,0,0
6542,NaN,NaN,0.0,1.0,Lake View,Entire rental unit,Entire home/apt,6,2.0,150.0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
data['review_scores_rating'].median()

4.84

In [48]:
#data.to_csv('data5.csv', index=False)

In [49]:
data.to_csv('data6.csv', index=False)